Given `demand.csv` and `supply.csv` as defined in the previous file, perform travel time lookups and output a final data file for import into the webapp

In [25]:
import pandas as pd 
demand_df = pd.read_csv("demand5.csv")
supply_df = pd.read_csv("supply5.csv")

In [26]:
import itertools 
import requests
import json

import psycopg2
import pandas as pd
from random import randint
import requests
import json

conn_string = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

def cartesian(df1, df2):
    rows = itertools.product(df1.iterrows(), df2.iterrows())
    df = pd.DataFrame(left.append(right) for (_, left), (_, right) in rows)
    return df.reset_index(drop=True)

def get_distance_duration_from_google_maps(source_lat, source_lng, dest_lat, dest_lng):
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins={source_lat},{source_lng}&destinations={dest_lat},{dest_lng}"\
    .format(**locals())
    
    r = requests.get(url)
    d = json.loads(r.content)
    
    return_dict = {}
    
    return_dict["distance_route_km"] = d["rows"][0]["elements"][0]["distance"]["value"]/1000.0
    return_dict["duration_min"] = d["rows"][0]["elements"][0]["duration"]["value"]/60.

    return return_dict

def get_distance_crow_flies(source_lat, source_lng, dest_lat, dest_lng):

    sql = """
        select st_distance(
            ST_Transform(ST_SetSRID(ST_MakePoint({source_lng}, {source_lat}), 4326), 27700),
            ST_Transform(ST_SetSRID(ST_MakePoint({dest_lng}, {dest_lat}), 4326), 27700)
        ) as distance
    """
    
    cur.execute(sql.format(**locals()))
    
    return {"distance_crowflies_km":  cur.fetchone()[0]/1000.0}

In [27]:
all_data = cartesian(demand_df, supply_df)

In [28]:
all_data.head()

,demand,demand_id,demand_lat,demand_lng,demand_name,supply,supply_id,supply_lat,supply_lng,supply_name
0,0.458502,0,52.594311,-0.138275,38 Willow,8.617344,0,52.554894,0.617282,Thetford
1,0.458502,0,52.594311,-0.138275,38 Willow,14.900903,1,54.080910,-1.459169,York
2,0.458502,0,52.594311,-0.138275,38 Willow,9.326675,2,53.753848,-0.673157,Wallingfen Ln
3,0.458502,0,52.594311,-0.138275,38 Willow,12.084897,3,52.502630,1.107698,Chapel View
4,0.458502,0,52.594311,-0.138275,38 Willow,15.587578,4,52.100816,1.393509,Woodbridge


In [29]:
def lat_lng_to_metrics(source_lat, source_lng, dest_lat, dest_lng):
    """
    Given source and dest coordinates, get travel time, route distance, 
    and distance as the crow flies and return as dict
    """
    d1 = get_distance_duration_from_google_maps(source_lat, source_lng, dest_lat, dest_lng)
    d2 = get_distance_crow_flies(source_lat, source_lng, dest_lat, dest_lng)
    d1.update(d2)
    return d1   

In [42]:
for r in all_data[860:].iterrows():
    index = r[0]
    row = r[1]
    
    metrics = lat_lng_to_metrics(row["demand_lat"], row["demand_lng"], row["supply_lat"], row["supply_lng"])
    
    for key, value in metrics.iteritems(): 
        all_data.loc[index, key] = value

(959, demand                   0.380609
 demand_id                     119
 demand_lat                51.8169
 demand_lng               -1.71263
 demand_name               Burford
 supply                    6.10625
 supply_id                       7
 supply_lat                52.4226
 supply_lng               -1.23701
 supply_name                United
 duration_min              86.6333
 distance_crowflies_km     74.8298
 distance_route_km           97.69
 Name: 959, dtype: object)

In [ ]:
all_data.sort("duration_min")

In [44]:
all_data.to_csv("all_data5.csv", index=False)

In [ ]:
print "\n".join(list(all_data.columns))